# Importing Python Libraries

In [1]:
import requests
import json
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [3]:
cur.execute("""SELECT Match_ID, TeamName, HomeTeam, AwayTeam, Date, Season, Div, 
                FTHG AS HomeTeamGoals, FTAG AS AwayTeamGoals, FTR AS Winners
                FROM Matches
                JOIN Teams_in_Matches
                USING (Match_ID)
                JOIN Unique_Teams
                USING (Unique_Team_ID)
                WHERE Season = 2011
                ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A


In [4]:
all_games = len(df)/2
non_German_div_index = df[df['Div'] == "E0" ].index
df.drop(non_German_div_index , inplace=True)
german_games = len(df)/2
print("{} games removed!".format(int(all_games - german_games)))

380 games removed!


In [5]:
for i, row in df.iterrows():
    if row.TeamName == row.AwayTeam:
        if row.Winners == 'A':
            df.loc[i, 'Result'] = 'W'
        if row.Winners == 'D':
            df.loc[i, 'Result'] = 'D'
        if row.Winners == 'H':
            df.loc[i, 'Result'] = 'L'
    else:
        if row.Winners == 'H':
            df.loc[i, 'Result'] = 'W'
        if row.Winners == 'D':
            df.loc[i, 'Result'] = 'D'
        if row.Winners == 'A':
            df.loc[i, 'Result'] = 'L'

In [6]:
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W


In [7]:
df_weather = df

In [8]:
for i, row in df.iterrows():
    if row.TeamName == row.AwayTeam:
        df.loc[i, 'GoalsScored'] = df.loc[i, 'AwayTeamGoals']
    elif row.TeamName == row.HomeTeam:
        df.loc[i, 'GoalsScored'] = df.loc[i, 'HomeTeamGoals']

In [9]:
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W,2.0
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L,1.0
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W,1.0


In [10]:
total_goals_scored = df.pivot_table(index = 'TeamName', aggfunc='sum')
total_goals_scored = total_goals_scored.rename(columns={"GoalsScored":"Total Goals Scored (2011 Season)"})
total_goals_scored = total_goals_scored.drop(columns = ['AwayTeamGoals','HomeTeamGoals','Match_ID','Season'])
total_goals_scored = total_goals_scored.reset_index()
total_goals_scored['Total Goals Scored (2011 Season)'].astype(int)
total_goals_scored.head()

,TeamName,Total Goals Scored (2011 Season)
0,Aachen,30.0
1,Augsburg,36.0
2,Bayern Munich,77.0
3,Bochum,41.0
4,Braunschweig,37.0


In [11]:
wins_and_losses = df.pivot_table(df, index = 'TeamName', columns=['Result'], aggfunc='count')
wins_and_losses = wins_and_losses.drop(columns = ['AwayTeam', 'AwayTeamGoals', 'Date', 'Div','HomeTeamGoals','Match_ID','Season','GoalsScored','HomeTeam'])
wins_and_losses.head()


Winners        
Result              D   L   W
TeamName                     
Aachen             13  15   6
Augsburg           14  12   8
Bayern Munich       4   7  23
Bochum              7  17  10
Braunschweig       15   9  10

In [12]:
wins_and_losses.columns = [col[1] for col in wins_and_losses.columns]
wins_and_losses.reset_index(inplace=True)
wins_and_losses.head(1)

,TeamName,D,L,W
0,Aachen,13,15,6


In [13]:
team_data = pd.concat([total_goals_scored, wins_and_losses], axis=1)
team_data = team_data.loc[:,~team_data.columns.duplicated()] # the ~ inverts the boolean statement from team_data.columns.duplicated() because wewant to keep non-duplicates
team_data.head()

,TeamName,Total Goals Scored (2011 Season),D,L,W
0,Aachen,30.0,13,15,6
1,Augsburg,36.0,14,12,8
2,Bayern Munich,77.0,4,7,23
3,Bochum,41.0,7,17,10
4,Braunschweig,37.0,15,9,10


In [14]:
# cur.execute("""SELECT *
#                 FROM Matches
#                 WHERE Season = 2011
#                 ;""")
# df_2 = pd.DataFrame(cur.fetchall())
# df_2.columns = [x[0] for x in cur.description]
# df_2.head()

In [15]:
# len(df_2)

In [16]:
# non_German_div_index = df_2[df_2['Div'] == "E0" ].index
# df_2.drop(non_German_div_index , inplace=True)

In [17]:
# len(df_2)

# Connecting to the DarkSky API

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [ ]:
keys = get_keys("/Users/ravimalde/.secret/dark_sky_api_2.json")
api_key = keys['api_key']

In [ ]:
latitude = 52.5200
longitude = 13.4050

date = "2020-01-30"
url = "https://api.darksky.net/forecast/{}/{},{},{}T16:00:00".format(api_key, latitude, longitude, date)
r = requests.get(url)

In [ ]:
r.json().keys()

In [ ]:
r.json()['daily']

In [ ]:
small_df = df_2.head(2)
small_df

In [ ]:
# latitude = 52.5200
# longitude = 13.4050

# for i, row in small_df.iterrows():
    
#     date = df.loc[i, 'Date']
#     url = "https://api.darksky.net/forecast/{}/{},{},{}T16:00:00".format(api_key, latitude, longitude, date)
#     r = requests.get(url)
# #     rain_probability = r.json()['currently']['precipProbability']
#     weather_summary = r.json()['daily']['data'][0]['summary']
# #     small_df['ChanceOfRain'] = rain_probability
#     small_df['Weather'] = weather_summary

In [ ]:
small_df

In [ ]:
len(df_2)

In [ ]:
# latitude = 52.5200
# longitude = 13.4050

# for i, row in df_2.iterrows():
    
#     date = df_2.loc[i, 'Date']
#     url = "https://api.darksky.net/forecast/{}/{},{},{}T16:00:00".format(api_key, latitude, longitude, date)
#     r = requests.get(url)
#     weather_summary = r.json()['daily']['data'][0].get("icon", "clear")
#     df_2.loc[i, 'Weather'] = weather_summary

In [ ]:
df_2.head()

In [ ]:
# df_2.to_csv("matches_with_weather.csv")

In [18]:
df_matches_with_weather = pd.read_csv("matches_with_weather.csv")

In [19]:
df_matches_with_weather.head()

,Unnamed: 0,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Weather
0,0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,rain
1,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,partly-cloudy-day
2,2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,partly-cloudy-day
3,3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,wind
4,4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,partly-cloudy-day


In [20]:
weather_dict = {}

for i, row in df_matches_with_weather.iterrows():
    weather_dict[df_matches_with_weather.loc[i, 'Match_ID']] = df_matches_with_weather.loc[i,'Weather']
    
weather_dict

{1092: 'rain',
 1093: 'partly-cloudy-day',
 1094: 'partly-cloudy-day',
 1095: 'wind',
 1096: 'partly-cloudy-day',
 1097: 'rain',
 1098: 'partly-cloudy-day',
 1099: 'partly-cloudy-day',
 1100: 'rain',
 1101: 'clear',
 1102: 'clear',
 1103: 'clear',
 1104: 'partly-cloudy-day',
 1105: 'clear',
 1106: 'clear',
 1107: 'rain',
 1108: 'partly-cloudy-day',
 1109: 'partly-cloudy-day',
 1110: 'rain',
 1111: 'partly-cloudy-day',
 1112: 'partly-cloudy-day',
 1113: 'partly-cloudy-day',
 1114: 'clear',
 1115: 'rain',
 1116: 'rain',
 1117: 'clear',
 1118: 'rain',
 1119: 'partly-cloudy-day',
 1120: 'partly-cloudy-day',
 1121: 'partly-cloudy-day',
 1122: 'partly-cloudy-day',
 1123: 'clear',
 1124: 'clear',
 1125: 'clear',
 1126: 'rain',
 1127: 'partly-cloudy-day',
 1128: 'clear',
 1129: 'rain',
 1130: 'rain',
 1131: 'clear',
 1132: 'partly-cloudy-day',
 1133: 'partly-cloudy-day',
 1134: 'wind',
 1135: 'rain',
 1136: 'partly-cloudy-day',
 1137: 'clear',
 1138: 'partly-cloudy-day',
 1139: 'partly-cloudy-

In [21]:
df_weather.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W,2.0
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L,1.0
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W,1.0


In [22]:
for i,row in df_weather.iterrows():
    for key,value in weather_dict.items():
        if row.Match_ID == key:
            df_weather.loc[i,'Weather'] = value

In [23]:
df_weather.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored,Weather
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0,rain
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0,rain
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W,2.0,partly-cloudy-day
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L,1.0,partly-cloudy-day
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W,1.0,partly-cloudy-day


In [24]:
df_weather['Weather'].unique()

array(['rain', 'partly-cloudy-day', 'wind', 'clear', 'clear-day',
       'cloudy'], dtype=object)

In [25]:
df_weather.drop(
    df_weather[df_weather['Weather'] != "rain"]
    .index, inplace=True)

In [26]:
df_weather.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored,Weather
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0,rain
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0,rain
10,1097,Bayern Munich,M'gladbach,Bayern Munich,2012-01-20,2011,D1,3,1,H,L,1.0,rain
11,1097,M'gladbach,M'gladbach,Bayern Munich,2012-01-20,2011,D1,3,1,H,W,3.0,rain
16,1100,Bayern Munich,Schalke 04,Bayern Munich,2011-09-18,2011,D1,0,2,A,W,2.0,rain


In [27]:
df_rainy_matches = df_weather
df_rainy_matches.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored,Weather
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0,rain
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0,rain
10,1097,Bayern Munich,M'gladbach,Bayern Munich,2012-01-20,2011,D1,3,1,H,L,1.0,rain
11,1097,M'gladbach,M'gladbach,Bayern Munich,2012-01-20,2011,D1,3,1,H,W,3.0,rain
16,1100,Bayern Munich,Schalke 04,Bayern Munich,2011-09-18,2011,D1,0,2,A,W,2.0,rain


In [28]:
rainy_wins_and_losses = df_rainy_matches.pivot_table(df, index = 'TeamName', columns=['Result'], aggfunc='count')
rainy_wins_and_losses = rainy_wins_and_losses.drop(columns = ['AwayTeam', 'AwayTeamGoals', 'Date', 'Div','HomeTeamGoals','Match_ID','Season','GoalsScored','HomeTeam','Weather'])
rainy_wins_and_losses.head()

Winners          
Result              D    L    W
TeamName                       
Aachen            4.0  1.0  2.0
Augsburg          3.0  NaN  3.0
Bayern Munich     NaN  2.0  5.0
Bochum            1.0  5.0  2.0
Braunschweig      1.0  1.0  4.0

In [29]:
rainy_wins_and_losses.columns = [col[1] for col in rainy_wins_and_losses.columns]
rainy_wins_and_losses.reset_index(inplace=True)
rainy_wins_and_losses.head(5)

,TeamName,D,L,W
0,Aachen,4.0,1.0,2.0
1,Augsburg,3.0,NaN,3.0
2,Bayern Munich,NaN,2.0,5.0
3,Bochum,1.0,5.0,2.0
4,Braunschweig,1.0,1.0,4.0


In [30]:
rainy_wins_and_losses['D'] = rainy_wins_and_losses['D'].fillna(0)
rainy_wins_and_losses['L'] = rainy_wins_and_losses['L'].fillna(0)
rainy_wins_and_losses['W'] = rainy_wins_and_losses['W'].fillna(0)
rainy_wins_and_losses.head()

,TeamName,D,L,W
0,Aachen,4.0,1.0,2.0
1,Augsburg,3.0,0.0,3.0
2,Bayern Munich,0.0,2.0,5.0
3,Bochum,1.0,5.0,2.0
4,Braunschweig,1.0,1.0,4.0


In [31]:
rainy_wins_and_losses['rainy_win_percentage'] = round((rainy_wins_and_losses['W']/(rainy_wins_and_losses['W']
                                                                             +rainy_wins_and_losses['D']+
                                                                             rainy_wins_and_losses['L']))*100, 1)

In [32]:
rainy_wins_and_losses.head()

,TeamName,D,L,W,rainy_win_percentage
0,Aachen,4.0,1.0,2.0,28.6
1,Augsburg,3.0,0.0,3.0,50.0
2,Bayern Munich,0.0,2.0,5.0,71.4
3,Bochum,1.0,5.0,2.0,25.0
4,Braunschweig,1.0,1.0,4.0,66.7


In [33]:
team_data = pd.concat([team_data, rainy_wins_and_losses], axis=1)
team_data = team_data.loc[:,~team_data.columns.duplicated()]
team_data.head()

,TeamName,Total Goals Scored (2011 Season),D,L,W,rainy_win_percentage
0,Aachen,30.0,13,15,6,28.6
1,Augsburg,36.0,14,12,8,50.0
2,Bayern Munich,77.0,4,7,23,71.4
3,Bochum,41.0,7,17,10,25.0
4,Braunschweig,37.0,15,9,10,66.7
